<div style="display: table; width: 100%;">
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 70%;">
    <h1>Inteligencia Artificial: Data Mining I</h1>
  </div>
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 30%;">
    <img src="https://github.com/UIDE-Tareas/4-Inteligencia-Artificial-Data-Mining-I-Tarea1/blob/main/Assets/UideLogo.png?raw=true" alt="Texto alternativo" style="width:50%;">
  </div>
</div>
<hr />

# Análisis Exploratorio de Datos
## Adult Income Dataset

**Maestría en Ciencia de Datos y Máquinas de Aprendizaje**  

**Autores - Estudiantes - Grupo 7:**

 💻 Luis Miguel Ramírez      
 💻 Aviles Paute José     
 💻 Espinoza Bone José    

### 🟦 Código fuente
[https://github.com/UIDE-Tareas/4-Inteligencia-Artificial-Data-Mining-I-Tarea1](https://github.com/UIDE-Tareas/4-Inteligencia-Artificial-Data-Mining-I-Tarea1)


**Fecha:** 29 de septiembre de 2025

---

### Objetivo
Realizar un análisis exploratorio completo del dataset Adult Income para identificar patrones, relaciones y factores que influyen en el ingreso de las personas (>50K vs <=50K).

In [ ]:
import sys
import subprocess
import os 
from pathlib import Path
from enum import Enum 
import zipfile
from typing import Optional

# Libs a instalar
LIBS = [
    "matplotlib"
    "requests",
    "numpy",
    "pandas",
    "seaborn", 
]

class ConsoleColor(Enum):
    RED = "\033[91m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    CYAN = "\033[96m"
    WHITE = "\033[97m"
    RESET = "\033[0m"

def PrintColor(message: str, color: ConsoleColor) -> str:
    RESET = ConsoleColor.RESET.value
    return f"{color.value}{message}{RESET}"


def ShowMessage(message: str, title: str, icon: str, color: ConsoleColor, sep:str = "\n"):
    colored_title = PrintColor(icon + f"  " + title.upper() + ":", color)
    print(f"{colored_title} {message}", sep= sep)


def ShowInfoMessage(message: str, title: str = "Info", sep:str = "\n"):
    ShowMessage(message, title, "ℹ️", ConsoleColor.CYAN, sep)


def ShowSuccessMessage(message: str, title: str = "Success", sep:str = "\n"):
    ShowMessage(message, title, "✅", ConsoleColor.GREEN, sep)


def ShowErrorMessage(message: str, title: str = "Error", sep:str = "\n"):
    ShowMessage(message, title, "❌", ConsoleColor.RED, sep)


def ShowWarningMessage(message: str, title: str = "Warning", sep:str = "\n"):
    ShowMessage(message, title, "⚠️", ConsoleColor.YELLOW, sep)

# Funcion para ejecutar comandos
def RunCommand(
    commandList: list[str], printCommand: bool = True, printError: bool = True
) -> subprocess.CompletedProcess:
    print("⏳", " ".join(commandList))
    stdOutput = None if printCommand else subprocess.DEVNULL
    errorOutput = None if printError else subprocess.PIPE
    result = subprocess.run(
        commandList, stdout=stdOutput, stderr=errorOutput, text=True
    )
    if result.returncode != 0 and printError:
        print(result.stderr)
    return result

# Función para instalar las dependencias
def InstallDeps(libs: Optional[list[str]] = None):
    print("ℹ️ Installing deps.")
    RunCommand(
        [sys.executable, "-m", "pip", "install", "--upgrade", "pip"], printCommand=True
    ) 
    if(libs is None or libs.count == 0):
        print("No hay elementos a instalar.")
    else:
        RunCommand([sys.executable, "-m", "pip", "install", *libs], printCommand=True)
        print("Deps installed.")
    print()

# Función para mostrar info el ambiente de ejecución
def ShowEnvironmentInfo():
    print("ℹ️  Environment Info:")
    print("Python Version:", sys.version)
    print("Platform:", sys.platform)
    print("Executable Path:", sys.executable)
    print("Current Working Directory:", os.getcwd())
    print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))
    print("sys.prefix:", sys.prefix)
    print("sys.base_prefix:", sys.base_prefix)
    print()

import requests

# Función para descargar un archivo
def DownloadFile(uri: str, filename: str, overwrite: bool = False, timeout: int = 20):
    dest = Path(filename).resolve()
    if dest.exists() and dest.is_file() and dest.stat().st_size > 0 and not overwrite:
        print(
            f'✅ Ya existe: "{dest}". No se descarga (use overwrite=True para forzar).'
        )
        return
    if dest.parent and not dest.parent.exists():
        dest.parent.mkdir(parents=True, exist_ok=True)
    print(f'ℹ️ Descargando "{uri}" → "{dest}"')
    try:
        with requests.get(uri, stream=True, timeout=timeout) as resp:
            resp.raise_for_status()
            tmp = dest.with_suffix(dest.suffix + ".part")
            with open(tmp, "wb") as f:
                for chunk in resp.iter_content(chunk_size=1024 * 64):
                    if chunk:  # filtra keep-alive chunks
                        f.write(chunk)
            tmp.replace(dest)
        print(f'✅ Archivo "{dest}" descargado exitosamente.')
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al descargar: {e}")


# Función para descomprimir un archivo zip
def UnzipFile(filename: str, outputDir: str):
    print(f'ℹ️ Descomprimiendo "{filename}" en "{outputDir}"')
    try:
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(outputDir)
        print(f"Descomprimido en: {os.path.abspath(outputDir)}")
    except Exception as e:
        print(f"Error: {e}")

InstallDeps(LIBS)
ShowEnvironmentInfo()

# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)



---
## 2. Descripción del Dataset

El dataset **Adult Income** (UCI Machine Learning Repository) contiene información censal con las siguientes características:

**Variables demográficas:** age, race, gender, native-country  
**Variables educativas:** education, educational-num  
**Variables laborales:** workclass, occupation, hours-per-week  
**Variables económicas:** fnlwgt, capital-gain, capital-loss  
**Variables sociales:** marital-status, relationship  
**Variable objetivo:** income (<=50K, >50K)

In [ ]:
DOWNLOAD_DIR = "Temp"
DATA_FILE_URI = "https://github.com/UIDE-Tareas/4-Inteligencia-Artificial-Data-Mining-I-Tarea1/raw/refs/heads/main/Data/adult.csv"
DATA_FILENAME = f"{DOWNLOAD_DIR}/adult.csv"
DownloadFile(DATA_FILE_URI, DATA_FILENAME)

# Cargar dataset
df = pd.read_csv(DATA_FILENAME)

print("="*70)
print("INFORMACIÓN GENERAL")
print("="*70)
print(f"\nDimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nColumnas: {list(df.columns)}")

# Vista previa
print("\n" + "="*70)
print("VISTA PREVIA DEL DATASET")
print("="*70)
display(df.head(10))

# Información estructural
print("\n" + "="*70)
print("ESTRUCTURA DEL DATASET")
print("="*70)
df.info()

# Calidad de datos
print("\n" + "="*70)
print("CALIDAD DE DATOS")
print("="*70)
print(f"Duplicados: {df.duplicated().sum():,}")
print(f"Valores faltantes totales: {df.isnull().sum().sum():,}")

if df.isnull().sum().sum() > 0:
    missing_df = pd.DataFrame({
        'Columna': df.columns,
        'Faltantes': df.isnull().sum(),
        'Porcentaje': (df.isnull().sum() / len(df) * 100).round(2)
    }).query('Faltantes > 0').sort_values('Faltantes', ascending=False)
    display(missing_df)
else:
    print("✓ Dataset completo sin valores faltantes")

---
## 3. Estadísticas Descriptivas

In [ ]:
# Variables numéricas
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print("="*70)
print("ESTADÍSTICAS - VARIABLES NUMÉRICAS")
print("="*70)

# Estadísticas completas
stats = df[num_cols].describe().T
stats['missing'] = df[num_cols].isnull().sum()
stats['skewness'] = df[num_cols].skew()
stats['kurtosis'] = df[num_cols].kurtosis()

display(stats.round(2))

# Interpretación de distribuciones
print("\nINTERPRETACIÓN DE ASIMETRÍA:")
for col in num_cols:
    skew = df[col].skew()
    if abs(skew) < 0.5:
        dist = "Simétrica"
    elif skew > 0:
        dist = "Sesgada positiva"
    else:
        dist = "Sesgada negativa"
    print(f"  {col:20s}: {dist:20s} (skew={skew:6.2f})")

In [ ]:
# Variables categóricas
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print("="*70)
print("ESTADÍSTICAS - VARIABLES CATEGÓRICAS")
print("="*70)

cat_summary = []
for col in cat_cols:
    cat_summary.append({
        'Variable': col,
        'Únicos': df[col].nunique(),
        'Más frecuente': df[col].mode()[0],
        'Frecuencia': df[col].value_counts().iloc[0],
        'Porcentaje': f"{df[col].value_counts(normalize=True).iloc[0]*100:.1f}%"
    })

display(pd.DataFrame(cat_summary))

# Variable objetivo
print("\n" + "="*70)
print("VARIABLE OBJETIVO: INCOME")
print("="*70)
income_dist = df['income'].value_counts()
income_pct = (income_dist / len(df) * 100).round(2)
income_df = pd.DataFrame({'Cantidad': income_dist, 'Porcentaje': income_pct})
display(income_df)

ratio = income_dist.max() / income_dist.min()
print(f"\nBalance del dataset: {ratio:.2f}:1 ({'DESBALANCEADO' if ratio > 1.5 else 'BALANCEADO'})")

---
## 4. Distribuciones Univariadas y Detección de Outliers

In [ ]:
# Histogramas y boxplots combinados
num_cols_main = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

fig, axes = plt.subplots(6, 2, figsize=(16, 20))
fig.suptitle('Distribuciones y Detección de Outliers - Variables Numéricas', 
             fontsize=16, fontweight='bold', y=0.995)

for idx, col in enumerate(num_cols_main):
    # Histograma (columna izquierda)
    ax_hist = axes[idx, 0]
    data = df[col].dropna()
    ax_hist.hist(data, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
    ax_hist.axvline(data.mean(), color='red', linestyle='--', linewidth=2, 
                    label=f'Media: {data.mean():.1f}')
    ax_hist.axvline(data.median(), color='green', linestyle='--', linewidth=2, 
                    label=f'Mediana: {data.median():.1f}')
    ax_hist.set_title(f'{col} - Distribución', fontweight='bold', fontsize=11)
    ax_hist.set_xlabel(col, fontweight='bold')
    ax_hist.set_ylabel('Frecuencia', fontweight='bold')
    ax_hist.legend(fontsize=8)
    ax_hist.grid(alpha=0.3)
    
    # Boxplot (columna derecha)
    ax_box = axes[idx, 1]
    bp = ax_box.boxplot(data, vert=True, patch_artist=True,
                         boxprops=dict(facecolor='lightblue'),
                         medianprops=dict(color='red', linewidth=2))
    ax_box.set_title(f'{col} - Boxplot', fontweight='bold', fontsize=11)
    ax_box.set_ylabel('Valor', fontweight='bold')
    ax_box.grid(alpha=0.3)
    ax_box.set_xticklabels([''])

plt.tight_layout()
plt.show()

# Análisis de outliers
print("="*70)
print("ANÁLISIS DE OUTLIERS (Método IQR)")
print("="*70)

outlier_data = []
for col in num_cols_main:
    Q1, Q3 = df[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)][col]
    
    outlier_data.append({
        'Variable': col,
        'Q1': Q1, 
        'Q3': Q3, 
        'IQR': IQR,
        'Límite Inf': lower, 
        'Límite Sup': upper,
        'Outliers': len(outliers),
        'Porcentaje': f"{len(outliers)/len(df)*100:.2f}%"
    })

display(pd.DataFrame(outlier_data))

In [ ]:
# Countplots
cat_cols_main = ['workclass', 'education', 'marital-status', 'occupation', 'race', 'gender']

fig, axes = plt.subplots(3, 2, figsize=(18, 14))
fig.suptitle('Distribución de Variables Categóricas', fontsize=16, fontweight='bold')

for idx, col in enumerate(cat_cols_main):
    ax = axes[idx//2, idx%2]
    vc = df[col].value_counts()
    vc.plot(kind='barh', ax=ax, color='steelblue', edgecolor='black')
    ax.set_title(col.upper(), fontweight='bold', fontsize=12)
    ax.set_xlabel('Frecuencia', fontweight='bold')
    ax.grid(alpha=0.3, axis='x')
    
    for i, v in enumerate(vc.values):
        ax.text(v+100, i, f'{v:,} ({v/len(df)*100:.1f}%)', va='center', fontsize=9)

plt.tight_layout()
plt.show()

---
## 5. Análisis Bivariado

In [ ]:
# Boxplots comparativos
num_key = ['age', 'educational-num', 'hours-per-week', 'capital-gain']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Variables Numéricas por Nivel de Ingreso', fontsize=16, fontweight='bold')

for idx, col in enumerate(num_key):
    ax = axes[idx//2, idx%2]
    data = [df[df['income']==cat][col].dropna() for cat in df['income'].unique()]
    bp = ax.boxplot(data, labels=df['income'].unique(), patch_artist=True, showmeans=True)
    
    for patch, color in zip(bp['boxes'], ['#ff6b6b', '#4ecdc4']):
        patch.set_facecolor(color)
    
    ax.set_title(col.upper(), fontweight='bold')
    ax.set_xlabel('Income', fontweight='bold')
    ax.set_ylabel(col, fontweight='bold')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas por grupo
print("="*70)
print("ESTADÍSTICAS POR GRUPO DE INGRESO")
print("="*70)
display(df.groupby('income')[num_key].describe().T.round(2))

In [ ]:
# Gráficos apilados
cat_key = ['gender', 'education', 'marital-status', 'occupation']

fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Variables Categóricas vs Income (% Normalizado)', fontsize=16, fontweight='bold')

for idx, col in enumerate(cat_key):
    ax = axes[idx//2, idx%2]
    ct = pd.crosstab(df[col], df['income'], normalize='index') * 100
    
    if '>50K' in ct.columns:
        ct = ct.sort_values('>50K', ascending=True)
    
    ct.plot(kind='barh', stacked=True, ax=ax, color=['#ff6b6b', '#4ecdc4'], edgecolor='black')
    ax.set_title(f'{col.upper()} vs INCOME', fontweight='bold')
    ax.set_xlabel('Porcentaje (%)', fontweight='bold')
    ax.legend(title='Income', loc='lower right')
    ax.grid(alpha=0.3, axis='x')
    ax.set_xlim(0, 100)

plt.tight_layout()
plt.show()

---
## 6. Matriz de Correlación

In [ ]:
# Matriz de correlación
corr = df.select_dtypes(include=[np.number]).corr()

fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, linewidths=1, cbar_kws={"label": "Correlación"}, vmin=-1, vmax=1, ax=ax)

ax.set_title('Matriz de Correlación (Pearson)', fontsize=14, fontweight='bold', pad=15)
plt.xticks(rotation=45, ha='right', fontweight='bold')
plt.yticks(rotation=0, fontweight='bold')
plt.tight_layout()
plt.show()

# Top correlaciones
pairs = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        pairs.append({
            'Var1': corr.columns[i],
            'Var2': corr.columns[j],
            'Correlación': corr.iloc[i,j]
        })

pairs_df = pd.DataFrame(pairs).sort_values('Correlación', key=abs, ascending=False)

print("="*70)
print("TOP 10 CORRELACIONES")
print("="*70)
display(pairs_df.head(10))

---
## 7. Hallazgos y Conclusiones

In [ ]:
print("="*70)
print("HALLAZGOS PRINCIPALES")
print("="*70)

# 1. Income
print("\n1. DISTRIBUCIÓN DE INGRESOS:")
for cat in df['income'].unique():
    count = (df['income']==cat).sum()
    pct = count/len(df)*100
    print(f"   {cat}: {count:,} ({pct:.1f}%)")

# 2. Género
print("\n2. DISPARIDAD DE GÉNERO:")
gender_income = pd.crosstab(df['gender'], df['income'], normalize='index')*100
for gender in df['gender'].unique():
    if '>50K' in gender_income.columns:
        print(f"   {gender}: {gender_income.loc[gender, '>50K']:.1f}% ganan >50K")

# 3. Educación
print("\n3. EDUCACIÓN:")
edu = df.groupby('education')['income'].apply(lambda x: (x=='>50K').sum()/len(x)*100).sort_values(ascending=False)
for e, p in edu.head(3).items():
    print(f"   {e}: {p:.1f}%")

# 4. Trabajo
print("\n4. HORAS TRABAJADAS:")
for cat in df['income'].unique():
    hrs = df[df['income']==cat]['hours-per-week'].mean()
    print(f"   Income {cat}: {hrs:.1f} horas/semana")

print("\n" + "="*70)
print("CONCLUSIONES")
print("="*70)
print("""
- Educación es el predictor más fuerte de ingresos altos
- Existe disparidad de género significativa
- Más horas trabajadas = mayores ingresos
- Estado civil "Married-civ-spouse" tiene mayor % de >50K
- Dataset desbalanceado requiere técnicas de balanceo
- Variables clave: educational-num, age, hours-per-week, marital-status
- Outliers en capital-gain/loss requieren tratamiento
""")
print("="*70)

---
## 8. Exportación de Gráficos a PDF

Generación automática de reporte visual con todos los gráficos del análisis.

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
import os

# Crear PDF
pdf_filename = f'Adult_Income_EDA_Graficos_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pdf'

print("="*70)
print("EXPORTANDO GRÁFICOS A PDF")
print("="*70)
print(f"\nGenerando: {pdf_filename}")

with PdfPages(pdf_filename) as pdf:
    
    # PÁGINA 1: Distribución de Income
    print("  [1/8] Distribución de Income...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle('Distribución de la Variable Objetivo: INCOME', fontsize=14, fontweight='bold')
    
    income_counts = df['income'].value_counts()
    income_pct = df['income'].value_counts(normalize=True) * 100
    
    income_counts.plot(kind='bar', ax=ax1, color=['#ff6b6b', '#4ecdc4'], edgecolor='black')
    ax1.set_title('Frecuencia Absoluta', fontweight='bold')
    ax1.set_xlabel('Income', fontweight='bold')
    ax1.set_ylabel('Cantidad', fontweight='bold')
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
    for i, v in enumerate(income_counts.values):
        ax1.text(i, v + 500, f'{v:,}', ha='center', fontweight='bold')
    
    ax2.pie(income_pct.values, labels=income_pct.index, autopct='%1.1f%%',
            startangle=90, colors=['#ff6b6b', '#4ecdc4'], explode=[0.05, 0.05])
    ax2.set_title('Porcentaje Relativo', fontweight='bold')
    
    plt.tight_layout()
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # PÁGINA 2: Histogramas
    print("  [2/8] Histogramas de variables numéricas...")
    num_cols_main = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']
    
    fig, axes = plt.subplots(3, 2, figsize=(12, 14))
    fig.suptitle('Distribuciones de Variables Numéricas', fontsize=14, fontweight='bold')
    
    for idx, col in enumerate(num_cols_main):
        ax = axes[idx//2, idx%2]
        data = df[col].dropna()
        ax.hist(data, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
        ax.axvline(data.mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {data.mean():.1f}')
        ax.axvline(data.median(), color='green', linestyle='--', linewidth=2, label=f'Mediana: {data.median():.1f}')
        ax.set_title(col, fontweight='bold')
        ax.set_xlabel(col, fontweight='bold')
        ax.set_ylabel('Frecuencia', fontweight='bold')
        ax.legend(fontsize=8)
        ax.grid(alpha=0.3)
    
    plt.tight_layout()
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # PÁGINA 3: Boxplots
    print("  [3/8] Boxplots de outliers...")
    fig, axes = plt.subplots(3, 2, figsize=(12, 14))
    fig.suptitle('Boxplots - Detección de Outliers', fontsize=14, fontweight='bold')
    
    for idx, col in enumerate(num_cols_main):
        ax = axes[idx//2, idx%2]
        bp = ax.boxplot(df[col].dropna(), vert=True, patch_artist=True,
                        boxprops=dict(facecolor='lightblue'),
                        medianprops=dict(color='red', linewidth=2))
        ax.set_title(col, fontweight='bold')
        ax.set_ylabel('Valor', fontweight='bold')
        ax.grid(alpha=0.3)
        ax.set_xticklabels([''])
    
    plt.tight_layout()
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # PÁGINA 4: Variables categóricas
    print("  [4/8] Distribuciones categóricas...")
    cat_cols_main = ['workclass', 'education', 'marital-status', 'occupation', 'race', 'gender']
    
    fig, axes = plt.subplots(3, 2, figsize=(12, 14))
    fig.suptitle('Distribuciones de Variables Categóricas', fontsize=14, fontweight='bold')
    
    for idx, col in enumerate(cat_cols_main):
        ax = axes[idx//2, idx%2]
        vc = df[col].value_counts()
        vc.plot(kind='barh', ax=ax, color='steelblue', edgecolor='black')
        ax.set_title(col.upper(), fontweight='bold')
        ax.set_xlabel('Frecuencia', fontweight='bold')
        ax.grid(alpha=0.3, axis='x')
        
        for i, v in enumerate(vc.values[:8]):
            if i < len(vc):
                ax.text(v+50, i, f'{v:,}', va='center', fontsize=8)
    
    plt.tight_layout()
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # PÁGINA 5: Income vs Numéricas
    print("  [5/8] Análisis bivariado - Income vs numéricas...")
    num_key = ['age', 'educational-num', 'hours-per-week', 'capital-gain']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('Variables Numéricas por Nivel de Ingreso', fontsize=14, fontweight='bold')
    
    for idx, col in enumerate(num_key):
        ax = axes[idx//2, idx%2]
        data = [df[df['income']==cat][col].dropna() for cat in df['income'].unique()]
        bp = ax.boxplot(data, labels=df['income'].unique(), patch_artist=True, showmeans=True)
        
        for patch, color in zip(bp['boxes'], ['#ff6b6b', '#4ecdc4']):
            patch.set_facecolor(color)
        
        ax.set_title(col.upper(), fontweight='bold')
        ax.set_xlabel('Income', fontweight='bold')
        ax.set_ylabel(col, fontweight='bold')
        ax.grid(alpha=0.3)
    
    plt.tight_layout()
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # PÁGINA 6: Income vs Categóricas
    print("  [6/8] Análisis bivariado - Income vs categóricas...")
    cat_key = ['gender', 'education', 'marital-status', 'occupation']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    fig.suptitle('Variables Categóricas vs Income (% Normalizado)', fontsize=14, fontweight='bold')
    
    for idx, col in enumerate(cat_key):
        ax = axes[idx//2, idx%2]
        ct = pd.crosstab(df[col], df['income'], normalize='index') * 100
        
        if '>50K' in ct.columns:
            ct = ct.sort_values('>50K', ascending=True).tail(10)
        
        ct.plot(kind='barh', stacked=True, ax=ax, color=['#ff6b6b', '#4ecdc4'], edgecolor='black')
        ax.set_title(f'{col.upper()} vs INCOME', fontweight='bold')
        ax.set_xlabel('Porcentaje (%)', fontweight='bold')
        ax.legend(title='Income', loc='lower right', fontsize=8)
        ax.grid(alpha=0.3, axis='x')
        ax.set_xlim(0, 100)
    
    plt.tight_layout()
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # PÁGINA 7: Matriz de Correlación
    print("  [7/8] Matriz de correlación...")
    fig, ax = plt.subplots(figsize=(12, 10))
    
    corr = df.select_dtypes(include=[np.number]).corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    
    sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, linewidths=1, cbar_kws={"label": "Correlación"}, 
                vmin=-1, vmax=1, ax=ax)
    
    ax.set_title('Matriz de Correlación (Pearson)', fontsize=14, fontweight='bold', pad=15)
    plt.xticks(rotation=45, ha='right', fontweight='bold')
    plt.yticks(rotation=0, fontweight='bold')
    
    plt.tight_layout()
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # PÁGINA 8: CONCLUSIONES
    print("  [8/8] Generando conclusiones...")
    fig = plt.figure(figsize=(11, 14))
    fig.patch.set_facecolor('white')
    ax = fig.add_subplot(111)
    ax.axis('off')
    
    # Calcular estadísticas
    income_ratio = df['income'].value_counts().max() / df['income'].value_counts().min()
    gender_stats = pd.crosstab(df['gender'], df['income'], normalize='index') * 100
    edu_top = df.groupby('education')['income'].apply(lambda x: (x=='>50K').sum()/len(x)*100).sort_values(ascending=False).head(3)
    hours_high = df[df['income']=='>50K']['hours-per-week'].mean()
    hours_low = df[df['income']=='<=50K']['hours-per-week'].mean()
    age_mean = df['age'].mean()
    marital_top = df.groupby('marital-status')['income'].apply(lambda x: (x=='>50K').sum()/len(x)*100).sort_values(ascending=False).head(1)
    
    conclusiones = f"""
ANÁLISIS EXPLORATORIO DE DATOS
Adult Income Dataset
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

HALLAZGOS PRINCIPALES

1. DISTRIBUCIÓN DE INGRESOS
   • Dataset desbalanceado con ratio {income_ratio:.2f}:1
   • {df['income'].value_counts().iloc[0]:,} personas ganan ≤50K
   • {df['income'].value_counts().iloc[1]:,} personas ganan >50K

2. DISPARIDAD DE GÉNERO
   • Masculino: {gender_stats.loc['Male', '>50K']:.1f}% ganan >50K
   • Femenino: {gender_stats.loc['Female', '>50K']:.1f}% ganan >50K
   • Brecha: {gender_stats.loc['Male', '>50K'] - gender_stats.loc['Female', '>50K']:.1f} puntos porcentuales

3. EDUCACIÓN
   • {edu_top.index[0]}: {edu_top.iloc[0]:.1f}%
   • {edu_top.index[1]}: {edu_top.iloc[1]:.1f}%
   • {edu_top.index[2]}: {edu_top.iloc[2]:.1f}%

4. HORAS TRABAJADAS
   • Income >50K: {hours_high:.1f} horas/semana
   • Income ≤50K: {hours_low:.1f} horas/semana

5. DEMOGRAFÍA
   • Edad promedio: {age_mean:.1f} años
   • Estado civil top: {marital_top.index[0]} ({marital_top.iloc[0]:.1f}%)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

CONCLUSIONES

- La EDUCACIÓN es el factor más determinante para ingresos altos
- Existe DISPARIDAD DE GÉNERO significativa
- HORAS TRABAJADAS correlacionan con mayores ingresos
- ESTADO CIVIL influye en probabilidad de ingresos >50K
- Variables clave: educational-num, age, hours-per-week
- Presencia de outliers en capital-gain y capital-loss

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Grupo 7: Luis Miguel Ramirez | Espinoza Bone Jose | Aviles Paute Jose
UIDE - Data Mining I - {datetime.now().strftime("%d/%m/%Y")}
"""
    
    ax.text(0.05, 0.95, conclusiones, transform=ax.transAxes,
            fontsize=10, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.2))
    
    plt.tight_layout()
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    
    # Metadata
    d = pdf.infodict()
    d['Title'] = 'Análisis Exploratorio - Adult Income Dataset'
    d['Author'] = 'Grupo 7'
    d['CreationDate'] = datetime.now()

print(f"\n✓ PDF generado: {pdf_filename}")
print(f"✓ Total páginas: 8")
print("="*70)